### 1. Import dei pachetti necessari

In [1]:
import numpy as np
import cv2 as cv
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import load_model

Using TensorFlow backend.
C:\Users\feder\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\feder\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\feder\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\feder\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

### 2. Caricamento dei modelli
- modello per il ricnoscimento del soggetto
- modello per il ricnoscimento della presenza/assenza della mascherina sul volto

In [2]:
model_subject = load_model("models_saved/vggface_people")
model_mask = load_model("models_saved/vggface_mask")


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



### 3. Demo live

In [4]:
face_detector = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv.VideoCapture(0)

cap.set(3,640)
cap.set(4,480)

while True:
    ret, frame = cap.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    color = (255,0,0)
    face = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    faces = face_detector.detectMultiScale(frame_gray,1.1,4)
    
    try:
        faces = faces.tolist()
        faces.sort()
    except:
        pass
    
    for (x,y,w,h) in faces:
        face = frame[y:y+h,x:x+w,:]
        face = cv.cvtColor(face, cv.COLOR_BGR2RGB)
        img_pixels = cv.resize(face, (224, 224)) 
        img_pixels = img_to_array(img_pixels)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels = img_pixels.reshape(1,224,224,3)
        img_pixels = preprocess_input(img_pixels)
        
        y_ana = np.argmax(model_subject.predict(img_pixels)[1], axis=1)[0]
        y_fede = np.argmax(model_subject.predict(img_pixels)[0], axis=1)[0]
        y_da = np.argmax(model_subject.predict(img_pixels)[2], axis=1)[0]


        #subject = str()


        if y_ana==1 and y_fede==0 and y_da==0:
            subject = 'Anastasia'
        elif y_ana==0 and y_fede==1 and y_da==0:
            subject = 'Federico'
        elif y_ana==0 and y_fede==0 and y_da==1:
            subject = 'Davide'
        else:
            subject = 'unknown'
        
        y_pred_mask = np.argmax(model_mask.predict(img_pixels), axis=1)[0]
        
        if y_pred_mask==0:
            mask="No"
        elif y_pred_mask==1:
            mask="Yes"
            
        
            
    for (x,y,w,h) in faces:
        
        face = frame[y:y+h,x:x+w,:]
        face = cv.cvtColor(face, cv.COLOR_BGR2RGB)
        
        if(mask=="Yes"):
            color=(0,255,0)
        else:
            color=(0,0,255)
            
        #cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        
        try:
            cv.rectangle(frame, (x, y), (x+w, y+h), color, 2)
           
            cv.putText(frame, 
                       ("PERSON: " + subject + " - MASK: "+mask), 
                       ((5, 30)), 
                        cv.FONT_HERSHEY_SIMPLEX,0.8,color,2)
        except:
            pass
    
    cv.imshow('FaceDetection', frame)
    
    k = cv.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break

cap.release()
cv.destroyAllWindows()        